In [1]:
# https://github.com/IShengFang/SpectralNormalizationKeras

In [2]:
# VGG and CLF only 
# 350
# cov loss
# fid2 loss ( CLF flatten(real) , CLF flatten(real) )
# z clf score
# hard VGG images
# disc1D BN and dropout
# remove MMD/fid 
# GaussianNoise only resblock down
# zmean->0 zvar->1 as loss 
# zscore bce
#unet

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Lambda, Input, GaussianNoise,concatenate, Dense, Dropout, Conv2D, Add, UpSampling2D, Dot, Conv2DTranspose, Activation, Reshape, InputSpec, LeakyReLU, Flatten, BatchNormalization
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K  
# from keras.utils.generic_utils import Progbar
from time import time
import pandas as pd

# from keras.engine import *
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
# from keras.utils.generic_utils import func_dump
# from keras.utils.generic_utils import func_load
# from keras.utils.generic_utils import deserialize_keras_object
# from keras.utils.generic_utils import has_arg
# from keras.utils import conv_utils
from keras.layers import Dense, Conv1D, Conv2D, Conv3D, Conv2DTranspose, Embedding
import tensorflow as tf
import cv2
import os
from imutils import paths
from keras.models import load_model
# import tensorflow_probability as tfp
from keras.applications.vgg19 import VGG19
# from tensorflow_addons.layers import SpectralNormalization
from keras.activations import swish
from keras.initializers import glorot_uniform, glorot_normal, GlorotUniform

2024-06-23 20:30:36.632649: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-23 20:30:37.700857: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
domain_path = '/sise/assafzar-group/assafzar/oded_R/DISCOVER/ALTZHEIMER_MRI/'

In [5]:
#Hyperperemeter
BATCHSIZE=64
LEARNING_RATE = 0.0002 # initial 0.0002
TRAINING_RATIO = 1
BETA_1 = 0.0
BETA_2 = 0.9
EPOCHS = 500
BN_MIMENTUM = 0.1
BN_EPSILON  = 0.00002 
latent_dim = 350
img_size = 64
channels = 1
img_shape = (img_size, img_size, channels)
nmax = 200000
ker_init = GlorotUniform(seed=None)
noise_var = 1

load_model_true = 1


In [6]:
# load data
margin=20
def load_images(directory):
    imagePaths = list(paths.list_images(directory))
    data = []
    images_name = []
    i = 1
    for imagePath in imagePaths:
        images_name.append(imagePath.split("/")[-1])
        image = cv2.imread(imagePath) # load the image
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) # swap color channels
        image = image[margin:image.shape[0]-margin , margin:image.shape[1]-margin] 
        image = cv2.resize(image, (img_size,img_size)) 
        data.append(image)
        i = i+1
        if i>nmax:
            break
    data_arr = np.array(data)
    req_images = np.array(data)
    req_images =  np.array(req_images).astype("float32")
    req_images = np.expand_dims(req_images, axis=3)
    return req_images , images_name

################################################################################################

# train
images1, names1 = load_images(domain_path + 'data/train/NonDemented') # ('/home/odedrot/images/ccf_images_64pix_train')
images0, names0 = load_images(domain_path + 'data/train/MildDemented') # ('/home/odedrot/images/Unet_images_64pix_train')  
images2, names2 = load_images(domain_path + 'data/train/VeryMildDemented') # ('/home/odedrot/images/ccf_images_64pix_train')

X_train = np.concatenate((images0, images1, images2))

################################################################################################
# test
images1, names1 = load_images(domain_path + 'data/test/NonDemented') # ('/home/odedrot/images/Unet_images_64pix_clf_bc_test')
images0, names0 = load_images(domain_path + 'data/test/MildDemented') # ('/home/odedrot/images/Unet_images_64pix_clf_aa_test')
images2, names2 = load_images(domain_path + 'data/test/VeryMildDemented') # ('/home/odedrot/images/Unet_images_64pix_clf_aa_test')

X_test = np.concatenate((images0, images1, images2)) 
names_test = np.concatenate((names0, names1, names2))

print('X_train' , X_train.shape)
print('X_test' , X_test.shape)




##################################################################################################

X_train (6101, 64, 64, 1)
X_test (299, 64, 64, 1)


In [7]:
def brightness_aug(img, b_val):
    img = img + b_val
    img = np.clip(img, 0, 1) 
    return img
def rotate_aug(img, angle):
    h, w = img_size, img_size
    center = (w / 2, h / 2)
    scale = 1
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(img, M, (w, h))
    return rotated
def flip_ver(img):
    flipVertical = cv2.flip(img, 0)
    return flipVertical
def flip_hor(img):
    flipHorizontal = cv2.flip(img, 1)
    return flipHorizontal
def noise_aug(img):
    original_img = np.copy(img)
    noise_mean = 0
    noise_var = 0.001
    noise_img = np.random.normal(noise_mean,noise_var, size=(img.shape[0], img.shape[0]))
    img = img + noise_img
    img = np.clip(img, 0, 1) 
    img[original_img<0.01] = 0 
    return img

def augment_img(img):
    aug_img = np.copy(img)
    # rotate
    rand_val = np.random.uniform(0,1)
    if rand_val > 0.5:
        angle = np.round(tf.random.shuffle(np.linspace(90, 270, num=3))[0].numpy(),3) # np.linspace(-15, 15, num=31
        aug_img = rotate_aug(aug_img, angle)
    # brightness
    rand_val = np.random.uniform(0,1)
    if rand_val > 0.5:
        b_val = np.round(tf.random.shuffle(np.linspace(-0.2, 0.2, num=100))[0].numpy(),3)
        aug_img = brightness_aug(aug_img, b_val)
    # flip hor
    rand_val = np.random.uniform(0,1)
    if rand_val > 0.5:
        aug_img = flip_hor(aug_img)
    # flip ver
    rand_val = np.random.uniform(0,1)
    if rand_val > 0.5:
        aug_img = flip_ver(aug_img)
    # noise
    rand_val = np.random.uniform(0,1)
    if rand_val > 0.5:
        aug_img = noise_aug(aug_img)
    return aug_img

def augment_imgs(imgs):
    aug_imgs = []
    for i in range(len(imgs)):
        aug_imgs.append(augment_img(imgs[i][:,:,0]))
    aug_imgs = np.array(aug_imgs)
    aug_imgs = np.expand_dims(aug_imgs, axis=3)
    return aug_imgs

In [8]:
def sample_images(imgs):
    r, c = 2, 10
    fig, axs = plt.subplots(r, c, figsize=(20,4))
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    plt.show()

In [9]:
class DenseSN(Dense):
    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(input_dim, self.units),
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        self.u = self.add_weight(shape=tuple([1, self.kernel.shape.as_list()[-1]]),
                                 initializer=initializers.RandomNormal(0, 1),
                                 name='sn',
                                 trainable=False)
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True
        
    def call(self, inputs, training=None):
        def _l2normalize(v, eps=1e-12):
            return v / (K.sum(v ** 2) ** 0.5 + eps)
        def power_iteration(W, u):
            _u = u
            _v = _l2normalize(K.dot(_u, K.transpose(W)))
            _u = _l2normalize(K.dot(_v, W))
            return _u, _v
        W_shape = self.kernel.shape.as_list()
        #Flatten the Tensor
        W_reshaped = K.reshape(self.kernel, [-1, W_shape[-1]])
        _u, _v = power_iteration(W_reshaped, self.u)
        #Calculate Sigma
        sigma=K.dot(_v, W_reshaped)
        sigma=K.dot(sigma, K.transpose(_u))
        #normalize it
        W_bar = W_reshaped / sigma
        #reshape weight tensor
        if training in {0, False}:
            W_bar = K.reshape(W_bar, W_shape)
        else:
            with tf.control_dependencies([self.u.assign(_u)]):
                 W_bar = K.reshape(W_bar, W_shape)  
        output = K.dot(inputs, W_bar)
        if self.use_bias:
            output = K.bias_add(output, self.bias, data_format='channels_last')
        if self.activation is not None:
            output = self.activation(output)
        return output
    

class ConvSN2D(Conv2D):

    def build(self, input_shape):
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                             'should be defined. Found `None`.')
        input_dim = input_shape[channel_axis]
        kernel_shape = self.kernel_size + (input_dim, self.filters)

        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)

        if self.use_bias:
            self.bias = self.add_weight(shape=(self.filters,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
            
        self.u = self.add_weight(shape=tuple([1, self.kernel.shape.as_list()[-1]]),
                         initializer=initializers.RandomNormal(0, 1),
                         name='sn',
                         trainable=False)
        
        # Set input spec.
        self.input_spec = InputSpec(ndim=self.rank + 2,
                                    axes={channel_axis: input_dim})
        self.built = True
    def call(self, inputs, training=None):
        def _l2normalize(v, eps=1e-12):
            return v / (K.sum(v ** 2) ** 0.5 + eps)
        def power_iteration(W, u):
            #Accroding the paper, we only need to do power iteration one time.
            _u = u
            _v = _l2normalize(K.dot(_u, K.transpose(W)))
            _u = _l2normalize(K.dot(_v, W))
            return _u, _v
        #Spectral Normalization
        W_shape = self.kernel.shape.as_list()
        #Flatten the Tensor
        W_reshaped = K.reshape(self.kernel, [-1, W_shape[-1]])
        _u, _v = power_iteration(W_reshaped, self.u)
        #Calculate Sigma
        sigma=K.dot(_v, W_reshaped)
        sigma=K.dot(sigma, K.transpose(_u))
        #normalize it
        W_bar = W_reshaped / sigma
        #reshape weight tensor
        if training in {0, False}:
            W_bar = K.reshape(W_bar, W_shape)
        else:
            with tf.control_dependencies([self.u.assign(_u)]):
                W_bar = K.reshape(W_bar, W_shape)
                
        outputs = K.conv2d(
                inputs,
                W_bar,
                strides=self.strides,
                padding=self.padding,
                data_format=self.data_format,
                dilation_rate=self.dilation_rate)
        if self.use_bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)
        if self.activation is not None:
            return self.activation(outputs)
        return outputs    

    
class ConvSN2DTranspose(Conv2DTranspose):

    def build(self, input_shape):
        if len(input_shape) != 4:
            raise ValueError('Inputs should have rank ' +
                             str(4) +
                             '; Received input shape:', str(input_shape))
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                             'should be defined. Found `None`.')
        input_dim = input_shape[channel_axis]
        kernel_shape = self.kernel_size + (self.filters, input_dim)

        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.filters,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
            
        self.u = self.add_weight(shape=tuple([1, self.kernel.shape.as_list()[-1]]),
                         initializer=initializers.RandomNormal(0, 1),
                         name='sn',
                         trainable=False)
        
        # Set input spec.
        self.input_spec = InputSpec(ndim=4, axes={channel_axis: input_dim})
        self.built = True  
    
    def call(self, inputs):
        input_shape = K.shape(inputs)
        batch_size = input_shape[0]
        if self.data_format == 'channels_first':
            h_axis, w_axis = 2, 3
        else:
            h_axis, w_axis = 1, 2

        height, width = input_shape[h_axis], input_shape[w_axis]
        kernel_h, kernel_w = self.kernel_size
        stride_h, stride_w = self.strides
        if self.output_padding is None:
            out_pad_h = out_pad_w = None
        else:
            out_pad_h, out_pad_w = self.output_padding

        # Infer the dynamic output shape:
        out_height = conv_utils.deconv_length(height,
                                              stride_h, kernel_h,
                                              self.padding,
                                              out_pad_h)
        out_width = conv_utils.deconv_length(width,
                                             stride_w, kernel_w,
                                             self.padding,
                                             out_pad_w)
        if self.data_format == 'channels_first':
            output_shape = (batch_size, self.filters, out_height, out_width)
        else:
            output_shape = (batch_size, out_height, out_width, self.filters)
            
        #Spectral Normalization    
        def _l2normalize(v, eps=1e-12):
            return v / (K.sum(v ** 2) ** 0.5 + eps)
        def power_iteration(W, u):
            #Accroding the paper, we only need to do power iteration one time.
            _u = u
            _v = _l2normalize(K.dot(_u, K.transpose(W)))
            _u = _l2normalize(K.dot(_v, W))
            return _u, _v
        W_shape = self.kernel.shape.as_list()
        #Flatten the Tensor
        W_reshaped = K.reshape(self.kernel, [-1, W_shape[-1]])
        _u, _v = power_iteration(W_reshaped, self.u)
        #Calculate Sigma
        sigma=K.dot(_v, W_reshaped)
        sigma=K.dot(sigma, K.transpose(_u))
        #normalize it
        W_bar = W_reshaped / sigma
        #reshape weight tensor
        if training in {0, False}:
            W_bar = K.reshape(W_bar, W_shape)
        else:
            with tf.control_dependencies([self.u.assign(_u)]):
                W_bar = K.reshape(W_bar, W_shape)
        self.kernel = W_bar
        
        outputs = K.conv2d_transpose(
            inputs,
            self.kernel,
            output_shape,
            self.strides,
            padding=self.padding,
            data_format=self.data_format)

        if self.use_bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)

        if self.activation is not None:
            return self.activation(outputs)
        return outputs
     

In [10]:
GaussianNoise_std = 0.03

def res_block_down(layer_input, filters):
    d = BatchNormalization()(layer_input)
    d = Activation('relu')(d)
    d = Conv2D(filters, kernel_size=3, strides=2, padding='same')(d)
    d = GaussianNoise(stddev=GaussianNoise_std)(d)
    d = BatchNormalization()(d)
    d = Activation('relu')(d)
    d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
    d = GaussianNoise(stddev=GaussianNoise_std)(d)
    d_res = Conv2D(filters, kernel_size=1, strides=2, padding='same')(layer_input)
    d = Add()([d, d_res])
    return d

def res_block_up(layer_input, filters):
    d = BatchNormalization()(layer_input)
    d = Activation('relu')(d)
    d = UpSampling2D(size=(2, 2), interpolation="nearest")(d)
    d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
    d = GaussianNoise(stddev=GaussianNoise_std)(d)
    d = BatchNormalization()(d)
    d = Activation('relu')(d)
    d = Conv2D(filters, kernel_size=3, strides=1, padding='same')(d)
    d = GaussianNoise(stddev=GaussianNoise_std)(d)
    d_res = UpSampling2D(size=(2, 2), interpolation="nearest")(layer_input)
    d_res = Conv2D(filters, kernel_size=1, strides=1, padding='same')(d_res)
    d = Add()([d, d_res])
    return d

def BuildEncoder():
    enc_input = Input(shape=img_shape, name='encoder_input') 
    X = Conv2D(64, kernel_size=3, padding='same', activation='relu')(enc_input)
    X = res_block_down(X, 128)
    X = res_block_down(X, 256)
    X = res_block_down(X, 512)
    X = res_block_down(X, 512)
    
    X = Flatten()(X)
    X = GaussianNoise(stddev=GaussianNoise_std)(X)
    X = swish(X) #############
        
    z = Dense(latent_dim)(X)
    z = GaussianNoise(stddev=GaussianNoise_std)(z)
    z = BatchNormalization()(z) # (None, 350)
    
    intermediate = tf.keras.layers.Reshape((350,1), input_shape=(350,))(z) # (None, 350, 1)
    first_half = tf.keras.layers.Cropping1D(cropping=(0,350-14))(intermediate) # (None, 14, 1)
    z_split = tf.keras.layers.Reshape((14,), input_shape=(14,1))(first_half) # (None, 14)
    
    # z_split = Lambda( lambda x: tf.split(x,num_or_size_splits=latent_dim//14,axis=1))(z) # list of vectors of size (None, 14) each

    clfscore = Dense(1, activation='sigmoid')(z_split) # z_split[0]
    
    encoder = Model(enc_input, [z, clfscore], name='encoder') # 
    
    return encoder

def BuildGenerator():
    dec_input = Input(shape=(latent_dim, ), name='decoder_input')
    last_conv_shape = (4, 4, 512)
    X = Dense(last_conv_shape[0] * last_conv_shape[1] * last_conv_shape[2])(dec_input)
    X = GaussianNoise(stddev=GaussianNoise_std)(X)
    X = Reshape((last_conv_shape[0], last_conv_shape[1], last_conv_shape[2]))(X)
    
    X = res_block_up(X, 512)
    X = res_block_up(X, 512)
    X = res_block_up(X, 256)
    X = res_block_up(X, 128)
    
    X = Conv2D(1, (3, 3), strides = (1,1), padding = 'same', kernel_initializer = ker_init)(X)
    X = Activation('sigmoid')(X)
    decoder = Model(dec_input, X, name='decoder')
    return decoder

In [11]:
# inputs = tf.keras.Input(shape=(500,)) # (None, 500)
# intermediate = tf.keras.layers.Dense(500,activation=tf.nn.relu)(inputs) # (None, 500)
# intermediate = tf.keras.layers.Reshape((500,1), input_shape=(500,))(intermediate) # (None, 500, 1)
# first_half = tf.keras.layers.Cropping1D(cropping=(0,200))(intermediate) # (None, 300, 1)
# first_half = tf.keras.layers.Reshape((300,), input_shape=(300,1))(first_half) # (None, 300)


In [12]:
def BuildDiscriminator1D():
    model = Sequential()
    model.add(Dense(2048, input_dim=latent_dim))
    model.add(BatchNormalization(scale=False))
    model.add(LeakyReLU(negative_slope=0.2))
    model.add((Dense(2048)))
    model.add(BatchNormalization(scale=False))
    model.add(LeakyReLU(negative_slope=0.2))
    model.add((Dense(2048)))
    model.add(BatchNormalization(scale=False))
    model.add(LeakyReLU(negative_slope=0.2))
    model.add((Dense(2048)))
    model.add(BatchNormalization(scale=False))
    model.add(LeakyReLU(negative_slope=0.2))
    model.add((Dense(2048)))
    model.add(LeakyReLU(negative_slope=0.2))
    model.add((Dense(2048)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))
    encoded_repr = Input(shape=(latent_dim, ))
    validity = model(encoded_repr)
    return Model(encoded_repr, validity, name='discriminator')

def build_recognizer():
    recognizer_input = Input(shape=img_shape, name='recognizer_input') # batchx128x128x1    
    x = Conv2D(32, 5, strides=2, padding="same", name="rec_conv1_preact")(recognizer_input)
    x = LeakyReLU(negative_slope=0.2)(x)
    x = Conv2D(64, 5, strides=2, padding="same", name="rec_conv2_preact")(x) 
    x = LeakyReLU(negative_slope=0.2)(x)
    x = Conv2D(128, 5, strides=2, padding="same", name="rec_conv3_preact")(x)  
    x = LeakyReLU(negative_slope=0.2)(x)
    x = Conv2D(256, 5, strides=2, padding="same", name="rec_conv4_preact")(x)  
    x = LeakyReLU(negative_slope=0.2)(x)
    
    x = Flatten()(x)
    x = Dense(512)(x)
    x = LeakyReLU(negative_slope=0.2)(x)
    recognizer_output = Dense(latent_dim, activation='softmax')(x)
    recognizer_value = Dense(1)(x)
    recognizer = Model(recognizer_input, recognizer_output, name="recognizer")
    return recognizer

In [13]:
def BuildUNet(): # https://github.com/zhixuhao/unet/blob/master/model.py
    inputs = Input(img_shape) 
    conv1i = Conv2D(64, 3, activation = 'relu', strides=1, padding = 'same', kernel_initializer = 'he_normal')(inputs) # 64
    conv1 = Conv2D(64, 3, activation = 'relu', strides=2, padding = 'same', kernel_initializer = 'he_normal')(conv1i) # 32
    conv2i = Conv2D(128, 3, activation = 'relu', strides=1, padding = 'same', kernel_initializer = 'he_normal')(conv1) # 32
    conv2 = Conv2D(128, 3, activation = 'relu', strides=2, padding = 'same', kernel_initializer = 'he_normal')(conv1) # 16
    conv3i = Conv2D(256, 3, activation = 'relu', strides=1, padding = 'same', kernel_initializer = 'he_normal')(conv2) # 16
    conv3 = Conv2D(256, 3, activation = 'relu', strides=2, padding = 'same', kernel_initializer = 'he_normal')(conv2) # 8
    conv4i = Conv2D(512, 3, activation = 'relu', strides=1, padding = 'same', kernel_initializer = 'he_normal')(conv3) # 8
    conv4 = Conv2D(512, 3, activation = 'relu', strides=2, padding = 'same', kernel_initializer = 'he_normal')(conv4i) # 4
    # middle
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4) # 4
    # up
    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv5)) # 8
    merge6 = concatenate([conv4i,up6], axis = 3) 
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3i,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2i,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1i,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    model = Model(inputs, conv10, name='unet')
    return model

In [14]:
def barlow_losses(z_a, z_b):
    z_a_norm = (z_a - tf.reduce_mean(z_a, axis=0)) #/ tf.math.reduce_std(z_a, axis=0)
    z_b_norm = (z_b - tf.reduce_mean(z_b, axis=0)) #/ tf.math.reduce_std(z_b, axis=0)
    cov_a_b = (tf.transpose(z_a_norm) @ z_b_norm) / z_a.shape[0] ############ 
    on_diag = tf.linalg.diag_part(cov_a_b)
    off_diag_mat = tf.linalg.set_diag(cov_a_b, tf.zeros(cov_a_b.shape[-1])  )
    abs_off_diag_mean = tf.reduce_sum(tf.abs(off_diag_mat)) / (cov_a_b.shape[0]**2 - cov_a_b.shape[0])
    off_diag_10_mat = off_diag_mat[0:10, 0:10]
    abs_off_diag_10_mean = tf.reduce_sum(tf.abs(abs_off_diag_mean)) / (off_diag_mat.shape[0]**2 - off_diag_mat.shape[0])
    # on_diag_loss = tf.reduce_sum( tf.pow(on_diag - 1, 2) ) # replace with hinge
    
    on_diag_loss = tf.reduce_mean( tf.nn.relu(1 - tf.sqrt(on_diag)) )
    
    off_diag_loss = abs_off_diag_mean
    a_b_loss = tf.reduce_mean(tf.abs(z_a_norm - z_b_norm)**2)
    return cov_a_b, on_diag_loss, off_diag_loss, a_b_loss, tf.reduce_mean(tf.abs(on_diag)), abs_off_diag_mean, abs_off_diag_10_mean

def get_off_diag_values(x):
    x_flat = tf.reshape(x,[-1])[:-1]
    x_reshape = tf.reshape(x_flat,[x.shape[0]-1, x.shape[0]+1])[:, 1:]
    off_diag_values = tf.reshape(x_reshape,[-1])
    return off_diag_values
def cov_loss_terms(z_batch):
    z_batch = z_batch - tf.reduce_mean(z_batch, axis=0)
    z_std = tf.sqrt(tf.math.reduce_variance(z_batch, axis=0) + 0.0001)
    z_std_loss = tf.reduce_mean( tf.nn.relu(1 - z_std) )
    cov = (tf.transpose(z_batch) @ z_batch) / (z_batch.shape[0] ) # cov = 1/N*(z.T*z)  tfp.stats.covariance(z_batch)
    diag_cov = tf.linalg.diag_part(cov) # vector of vars (z_std**2)  positives
    diag_cov_mean = tf.reduce_mean(tf.abs(diag_cov))    
    off_diag_loss    = tf.reduce_mean(tf.abs( get_off_diag_values(cov) ))
    off_diag_mean_10 = tf.reduce_mean(tf.abs(get_off_diag_values(cov[0:10,0:10])))
    return cov, z_std_loss, diag_cov_mean , off_diag_loss, off_diag_mean_10 
def calc_cov_loss(y_true, ypred):
    cov, z_std_loss, diag_cov_mean , off_diag_loss, off_diag_mean_10 = cov_loss_terms(y_true)
    cov_loss = 0.5*diag_cov_mean + 0.5*z_std_loss
    return cov_loss

def model_perc_loss_wrapper(model):
    def model_perc_loss(y_true, y_pred):
        # weights = [1,1,1]
        img_outputs = model( y_true ) 
        recon_img_outputs = model( y_pred ) 
        layers_losses = []
        for i in range(len(recon_img_outputs)):
            mae_losses = K.abs(img_outputs[i] - recon_img_outputs[i])
            layers_losses.append(K.mean(mae_losses))
        avg_loss = K.mean(tf.convert_to_tensor(layers_losses))
        return avg_loss
    return model_perc_loss

def z_mean_var_loss(y_true, y_pred):
    loss = K.abs(K.var(y_pred)-1) + K.abs(K.mean(y_pred))
    return K.mean(loss)

def perturbate_embedding(z_image_batch, batch_size):
    GT_onehot_vecs = np.zeros((batch_size, latent_dim)) # vectors of zeros which will get 1 in a random dim
    pert_onehot_vecs = np.copy(GT_onehot_vecs) # vectors of zeros which will get a value in a random dim , it is added to z in the graph
    for i in range(batch_size):
        pert_dim = np.random.randint(latent_dim, size=1)[0] # choose a dim
        values = np.linspace(-1.5, 1.5, num=16)
        pert_value = tf.random.shuffle(values)[0] # choose a value from values
        pert_onehot_vecs[i][pert_dim] = pert_value # change z to selected value
        GT_onehot_vecs[i][pert_dim] = 1 # put value 1 in one hot
    pert_onehot_vecs = np.array(pert_onehot_vecs)
    GT_onehot_vecs = np.array(GT_onehot_vecs)
    return pert_onehot_vecs, GT_onehot_vecs


In [15]:
encoder = BuildEncoder()
generator = BuildGenerator()
recognizer = build_recognizer()
unet = BuildUNet()
discriminator1D = BuildDiscriminator1D()

# ae_saved_epoch = 'vgg_seed_3_epoch_4_auc_0.79_gmax_0.73_tpr_0.79_tnr_0.68_acc_0.72_f1_0.72'

# ae_model_path = '/home/odedrot/saved_models_VAEGAN/AAE_opt6_retrain2/' + ae_saved_epoch + '.h5'
# ae_model = load_model(ae_model_path , compile=False, custom_objects={'ConvSN2D': ConvSN2D , 'DenseSN': DenseSN})
# encoder = ae_model.get_layer('encoder')
# generator = ae_model.get_layer('decoder')
# unet = ae_model.get_layer('unet')
# recognizer = ae_model.get_layer('recognizer')
# discriminator1D = ae_model.get_layer('discriminator')

import keras
@keras.saving.register_keras_serializable(package="my_package", name="gray2rgb")
def gray2rgb(tensor):
    tensor = tf.image.grayscale_to_rgb(tensor)
    return tensor

img = Input(shape=img_shape)
z_img, z_img_score = encoder(img) 
recon_img = generator(z_img)
recon_img_rescaled =  Lambda(gray2rgb)(recon_img)
                    
superimg = unet(recon_img)
superimg_rescaled = Lambda(gray2rgb)(superimg)
 

# CNN_saved_name = 'vgg_seed_3_epoch_33_auc_0.94_gmax_0.85_tpr_0.86_tnr_0.84.h5'
CNN_saved_name = 'vgg_seed_3_epoch_64_auc_0.75_gmax_0.73_tpr_0.69_tnr_0.78_acc_0.73_f1_0.72.keras' # new

CNN_clf_model_path = domain_path + 'saved_CLF_org/' + CNN_saved_name
CNN_clf_model = load_model(CNN_clf_model_path , compile=True, custom_objects={'Sigmoid':tf.keras.models.Model})
CNN_clf_model.trainable = False # we dont train the CNN
recon_img_clfscore = CNN_clf_model( recon_img_rescaled ) 

CNN_clf_flatten = Model(inputs=CNN_clf_model.input, outputs = CNN_clf_model.get_layer('flatten').output)
CNN_clf_flatten.trainable = False # we dont train the CNN

clf_outputs = [
               # CNN_clf_model.get_layer('block1_conv1').output,
               # CNN_clf_model.get_layer('block1_conv2').output,
               # CNN_clf_model.get_layer('block2_conv1').output,
               # CNN_clf_model.get_layer('block2_conv2').output,
               CNN_clf_model.get_layer('block3_conv1').output,
               CNN_clf_model.get_layer('block3_conv2').output,
               CNN_clf_model.get_layer('block3_conv3').output,
               CNN_clf_model.get_layer('block4_conv1').output,
               CNN_clf_model.get_layer('block4_conv2').output,
               CNN_clf_model.get_layer('block4_conv3').output,
               CNN_clf_model.get_layer('block5_conv1').output,
               CNN_clf_model.get_layer('block5_conv2').output,
               CNN_clf_model.get_layer('block5_conv3').output,
               CNN_clf_model.get_layer('flatten').output,
               CNN_clf_model.get_layer('dense').output,]
                  
CNN_clf_Model_outputs = Model(inputs=CNN_clf_model.input, outputs = clf_outputs)
CNN_clf_Model_outputs.trainable = False # we dont train the CNN

VGG_model = VGG19(weights="imagenet", include_top=False, input_shape=(img_size, img_size,3)) # ResNet50 , VGG16, VGG19
VGG_outputs = [
               # VGG_model.get_layer('block1_conv1').output,   
               # VGG_model.get_layer('block1_conv2').output,
               # VGG_model.get_layer('block2_conv1').output,
               # VGG_model.get_layer('block2_conv2').output,   
               VGG_model.get_layer('block3_conv1').output,   
               VGG_model.get_layer('block3_conv2').output,
               VGG_model.get_layer('block3_conv3').output,
               VGG_model.get_layer('block4_conv1').output,
               VGG_model.get_layer('block4_conv2').output,
               VGG_model.get_layer('block4_conv3').output,
               VGG_model.get_layer('block4_conv4').output,
               VGG_model.get_layer('block5_conv1').output,
               VGG_model.get_layer('block5_conv2').output,
               VGG_model.get_layer('block5_conv3').output,
               VGG_model.get_layer('block5_conv4').output,]
VGG_Model_outputs = Model(inputs=VGG_model.input, outputs = VGG_outputs)
VGG_Model_outputs.trainable = False # we dont train the CNN

# recognizer
recognizer.compile(loss='categorical_crossentropy', optimizer=Adam(LEARNING_RATE, beta_1=BETA_1, beta_2=BETA_2), metrics=['accuracy'])
pert_vec = Input(shape=(latent_dim,))
z_img_p = Add()([z_img, pert_vec])
generator.trainable = False
pert_img = generator(z_img_p)
generator.trainable = True
onehot_dim_pred = recognizer(pert_img - recon_img)

# gen images for fid2
# noise = Input(shape=(latent_dim,))
# gen_img = generator(noise)

# disc1D 

discriminator1D.compile(loss='bce', optimizer=Adam(LEARNING_RATE/10, beta_1=BETA_1, beta_2=BETA_2))
discriminator1D.trainable = False 
disc1D_output = discriminator1D(z_img)

AE = Model([img, pert_vec], [recon_img_rescaled,recon_img_rescaled, superimg_rescaled, superimg_rescaled, z_img, z_img, z_img_score, disc1D_output, onehot_dim_pred])


2024-06-23 20:30:58.017142: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/odedrot/.conda/envs/DISCOVERtmp/lib/python3.9/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/odedrot/.conda/envs/DISCOVERtmp/lib/python3.9/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


In [16]:
#               VGG, clf, unetVGG, unetCLF,      calc_cov_loss, z_mean_var_loss,   'mse',  'bce',   'categorical_crossentropy'
loss_weights = [2,    2,    4,        4,             0.0000001,     0.00000001,      10,     1,       1]
AE.compile(loss=[model_perc_loss_wrapper(VGG_Model_outputs), model_perc_loss_wrapper(CNN_clf_Model_outputs), model_perc_loss_wrapper(VGG_Model_outputs), model_perc_loss_wrapper(CNN_clf_Model_outputs), calc_cov_loss, z_mean_var_loss, 'bce', 'bce', 'categorical_crossentropy'], optimizer=Adam(LEARNING_RATE, beta_1=BETA_1, beta_2=BETA_2), loss_weights=loss_weights)


In [ ]:
test_batch_size = 200
test_images = X_test[np.random.randint(0, X_test.shape[0], test_batch_size)] / 255 
test_images3D = test_images.repeat(3, -1)
test_images_scores = CNN_clf_model( test_images3D )
test_noise = np.random.normal(0,noise_var, size=(test_batch_size, latent_dim))
test_real_y = 0.9*np.ones((test_batch_size, 1))
test_fake_y = 0.1*np.ones((test_batch_size, 1))

real_y = 0.9*np.ones((BATCHSIZE, 1))
fake_y = 0.1*np.ones((BATCHSIZE, 1))

for epoch in range(EPOCHS):
    print("epoch", epoch)
    iteration = 0
    for n_batch in range(len(X_train) // BATCHSIZE):
        idx = np.random.randint(0, X_train.shape[0], 2*BATCHSIZE)
        image_batch = np.copy(X_train[idx]) 
        image_batch = image_batch / 255 
        
        ## choose high perc loss images
        recon_image_batch = generator.predict(encoder.predict(image_batch)[0], verbose = None) 
        samples_perc_loss = []
        for i in range(2*BATCHSIZE):
            samples_perc_loss.append( model_perc_loss_wrapper(VGG_Model_outputs)(image_batch[i:i+1].repeat(3, -1), recon_image_batch[i:i+1].repeat(3, -1)) )
        samples_perc_loss = np.array(samples_perc_loss)
        sorted_ind = np.argsort(samples_perc_loss)[::-1]
        # sorted_samples_perc_loss = samples_perc_loss[sorted_ind]
        # sorted_samples_perc_loss = sorted_samples_perc_loss[~np.isnan(sorted_samples_perc_loss)] # remove nan
        sorted_images = image_batch[sorted_ind]
        image_batch = sorted_images[0:BATCHSIZE]
        ##
        
        image_batch3D = image_batch.repeat(3, -1) 
        imgs_clfscore = CNN_clf_model( image_batch3D ) 
        z_image_batch = encoder.predict(image_batch, verbose = None)[0]
        discriminator1D.trainable = False
        
        # recognizer       
        pert_onehot_vecs, GT_onehot_vecs = perturbate_embedding(z_image_batch, BATCHSIZE)
        
        AE_loss = AE.train_on_batch([image_batch, pert_onehot_vecs], [image_batch3D, image_batch3D, image_batch3D, image_batch3D, z_image_batch,z_image_batch, imgs_clfscore, real_y, GT_onehot_vecs])
    
        # discriminator1D
        # discriminator1D
        for i in range(3):  # fake (z)
            discriminator1D.trainable = True 
            idx = np.random.randint(0, X_train.shape[0], BATCHSIZE)
            imgs = np.copy(X_train[idx]) 
            imgs = imgs / 255 
            z_images = encoder.predict(imgs, verbose = None)[0]
            d_loss = discriminator1D.train_on_batch(z_images, fake_y[0:BATCHSIZE] )
        for i in range(3): # real (n)
            discriminator1D.trainable = True 
            noise_vecs = np.random.normal(0,noise_var, size=(BATCHSIZE, latent_dim))
            d_loss = discriminator1D.train_on_batch(noise_vecs, real_y[0:BATCHSIZE] )
            
        # if iteration % 1 == 0: 
        #     discriminator1D.trainable = True 
        #     idx = np.random.randint(0, X_train.shape[0], BATCHSIZE)
        #     imgs = np.copy(X_train[idx]) 
        #     imgs = imgs / 255 
        #     z_images = encoder.predict(imgs)[0]
        #     noise_vecs = np.random.normal(0,noise_var, size=(BATCHSIZE, latent_dim))
        #     d_loss = discriminator1D.train_on_batch(np.concatenate((z_images, noise_vecs)), np.concatenate((fake_y[0:BATCHSIZE], real_y[0:BATCHSIZE])) )
###################################################################################################  
        # test
        iteration = iteration + 1
        if epoch % 10 == 0 and iteration % 90 == 0: 
            print('iteration ' , iteration)
            test_z_images, test_z_images_score  = encoder.predict(test_images, verbose = None)
            test_recon_images = generator.predict(test_z_images)
            test_z_recon_images, test_z_recon_images_score = encoder.predict(test_recon_images, verbose = None)
            test_generated_images = generator.predict(test_noise, verbose = None)
            test_z_generated_images = encoder.predict(test_generated_images, verbose = None)[0]
            
            test_super_images = unet.predict(test_recon_images, verbose = None)
            
            test_d1D_loss_real = discriminator1D.test_on_batch(test_noise, test_real_y)
            test_d1D_loss_fake = discriminator1D.test_on_batch(test_z_images, test_fake_y)
            
            ##
            test_Perc_loss = np.round( model_perc_loss_wrapper(VGG_Model_outputs)(test_images.repeat(3, -1), test_recon_images.repeat(3, -1)).numpy()  , 3)  
            test_CLF_loss = np.round( model_perc_loss_wrapper(CNN_clf_Model_outputs)(test_images.repeat(3, -1), test_recon_images.repeat(3, -1)).numpy()  , 3)  
            test_Super_Perc_loss = np.round( model_perc_loss_wrapper(VGG_Model_outputs)(test_images.repeat(3, -1), test_super_images.repeat(3, -1)).numpy()  , 3)  
            test_Super_CLF_loss = np.round( model_perc_loss_wrapper(CNN_clf_Model_outputs)(test_images.repeat(3, -1), test_super_images.repeat(3, -1)).numpy()  , 3)   
            
            ## CLF scores
            test_recon_images_scores = CNN_clf_model(test_recon_images.repeat(3, -1))
            test_super_images_scores = CNN_clf_model(test_super_images.repeat(3, -1))

            
            ## disentanglment
            test_pert_onehot_vecs, test_GT_onehot_vecs = perturbate_embedding(test_z_images, test_batch_size)
            test_recon_pert_imgs = generator.predict(test_z_images + test_pert_onehot_vecs, verbose = None)
            test_diff_recon_imgs = test_recon_pert_imgs - test_recon_images
            
            test_AE_loss = AE.test_on_batch([test_images, test_pert_onehot_vecs], [test_images3D, test_images3D, test_images3D, test_images3D, test_z_images,test_z_images, test_images_scores, test_real_y, test_GT_onehot_vecs])
            test_recognizer_output = recognizer.test_on_batch(test_diff_recon_imgs, test_GT_onehot_vecs)
            test_recognizer_loss = test_recognizer_output[0].item()
            test_recognizer_acc = test_recognizer_output[1].item()
            
            zscore_loss = np.round(np.mean(np.abs(test_z_images_score[:,0] - test_images_scores[:,0].numpy())**1),4)
            test_cov_var , test_cov_mean = cov_loss_terms(test_z_images)[2].numpy() , cov_loss_terms(test_z_images)[3].numpy()
            test_cov_loss = 1 - test_cov_var + test_cov_mean
           
            print('Disc1D_real' , test_d1D_loss_real)
            print('Disc1D_fake' , test_d1D_loss_fake)
            print('Perc_loss' , test_Perc_loss)
            print('CLF_loss' , test_CLF_loss)
            print('super Perc_loss' , test_Super_Perc_loss)
            print('super CLF_loss' , test_Super_CLF_loss)
            print('cov [1,0]' , test_cov_var , test_cov_mean)
            print( '[z_var, z_mean]' , np.var(test_z_images) , np.mean(test_z_images) )
            print('zscore_loss' , zscore_loss)
            print( 'recognizer_loss',  np.round(test_recognizer_loss,5),  np.round(test_recognizer_acc,5) )
            print('')
            # print('z_mmd_metric_2', z_mmd_loss2(test_z_images.astype('float32'), test_noise.astype('float32')).numpy())
            # print('MMD gen metric' , dist_sim_wrapper(CNN_clf_flatten)(test_generated_images, test_images).numpy() )
            # print('MMD recon metric' , dist_sim_wrapper(CNN_clf_flatten)(test_recon_images, test_images).numpy() )
            print('mae metric' , np.mean(np.abs(test_recon_images - test_images)) )
            print('score mae metric' , np.mean(np.abs(test_images_scores - test_recon_images_scores )) )
            print('mae(z,zrecon) metric' , np.mean(np.abs(test_z_recon_images - test_z_images)) )
            print('mae(noise,z_gen) metric' , np.mean(np.abs(test_z_generated_images - test_noise)) )
         
            print('')
            print('real')
            sample_images(test_images)
            print('')
            print('recon')
            sample_images(test_recon_images)
            print('')
            # print('recon')
            # sample_images(test_super_images)
            # print('')
            print('gen')
            sample_images(test_generated_images)
            # show gen imgs with disc1D scores close to 0 (more close to z)
            # noise_vecs_disc1D_pred = discriminator1D.predict(test_noise)[:,0]
            # sorted_ind = np.argsort(noise_vecs_disc1D_pred) # [::-1] 
            # sorted_noise_vecs_disc1D_pred = noise_vecs_disc1D_pred[sorted_ind]
            # sorted_test_noise = test_noise[sorted_ind] 
            # sorted_test_gen_images = generator.predict(sorted_test_noise)
            # sample_images(sorted_test_gen_images)
            # print('')
            # print('gen clf-->1')
            # show gen imgs with clf scores close to 1 
            # test_gen_imgs_clfscore = CNN_clf_model.predict(  test_generated_images.repeat(3, -1) )[:,0]
            # sorted_ind = np.argsort(test_gen_imgs_clfscore)[::-1] 
            # sample_images(test_generated_images[sorted_ind])
            # # generated scores dsitribution
            # _ = plt.hist(test_gen_imgs_clfscore.flatten(), 100)
            # plt.title('gen imgs clf scores')
            # plt.show()
            
            df = pd.concat([pd.DataFrame(test_z_images) , pd.DataFrame(test_images_scores)] , axis=1).reset_index()
            df = df.drop( ['index'] , axis=1)
            cor = df.corr()
            plt.figure(figsize=(30,10))
            plt.bar(np.array(df.columns[:-1]), cor.iloc[-1].values[:-1] , width=1) # normalized
            plt.xlabel("latent dimension", fontsize=50)
            plt.ylabel("correlation ", fontsize=50)
            plt.xticks(fontsize=30)
            plt.yticks(fontsize=30)
            plt.show()
            print(np.abs(cor.iloc[-1].values[15:-1]).max())
            print('')
            z_imgs_ = test_z_images - np.mean(test_z_images, axis=0)
            z_std = np.sqrt(np.var(z_imgs_, axis=0))
            cov = (z_imgs_.T @ z_imgs_) / (z_imgs_.shape[0] )
            plt.imshow(np.abs(cov[0:30, 0:30]))
            plt.show()
                        
            ## save
            encoder.save(         domain_path + 'saved_DISCOVER_MODEL/encoder_epoch_'  + str(epoch+1) + '_perc_' + str(test_Perc_loss) + '_CLF_' + str(test_CLF_loss) + '_zcr_' + str(zscore_loss) + '_disent_acc_' + str(np.round(test_recognizer_loss,3)) + '_disent_loss_' + str(np.round(test_recognizer_acc,5)) + '_cov_loss_' + str(np.round(test_cov_loss,5))   + '.keras', include_optimizer=False)
            generator.save(       domain_path + 'saved_DISCOVER_MODEL/generator_epoch_'  + str(epoch+1) + '_perc_' + str(test_Perc_loss) + '_CLF_' + str(test_CLF_loss) + '_zcr_' + str(zscore_loss) + '_disent_acc_' + str(np.round(test_recognizer_loss,3)) + '_disent_loss_' + str(np.round(test_recognizer_acc,5)) + '_cov_loss_' + str(np.round(test_cov_loss,5))   + '.keras', include_optimizer=False)
            discriminator1D.save( domain_path + 'saved_DISCOVER_MODEL/discriminator1D_epoch_'  + str(epoch+1) + '_perc_' + str(test_Perc_loss) + '_CLF_' + str(test_CLF_loss) + '_zcr_' + str(zscore_loss) + '_disent_acc_' + str(np.round(test_recognizer_loss,3)) + '_disent_loss_' + str(np.round(test_recognizer_acc,5)) + '_cov_loss_' + str(np.round(test_cov_loss,5))   + '.keras', include_optimizer=False)
            recognizer.save(      domain_path + 'saved_DISCOVER_MODEL/recognizer_epoch_'  + str(epoch+1) + '_perc_' + str(test_Perc_loss) + '_CLF_' + str(test_CLF_loss) + '_zcr_' + str(zscore_loss) + '_disent_acc_' + str(np.round(test_recognizer_loss,3)) + '_disent_loss_' + str(np.round(test_recognizer_acc,5)) + '_cov_loss_' + str(np.round(test_cov_loss,5))   + '.keras', include_optimizer=False)
            unet.save(            domain_path + 'saved_DISCOVER_MODEL/unet_epoch_'  + str(epoch+1) + '_perc_' + str(test_Perc_loss) + '_CLF_' + str(test_CLF_loss) + '_zcr_' + str(zscore_loss) + '_disent_acc_' + str(np.round(test_recognizer_loss,3)) + '_disent_loss_' + str(np.round(test_recognizer_acc,5)) + '_cov_loss_' + str(np.round(test_cov_loss,5))   + '.keras', include_optimizer=False)


epoch 0
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 175ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 177ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 175ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 177ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 177ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 177ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 189ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 176ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 184ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 187ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 169ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step
4/4 ━━━━━━━━━━━━━

In [19]:
len(X_train) // BATCHSIZE

95

In [18]:
X_train.shape

(6101, 64, 64, 1)

In [20]:
BATCHSIZE

64